In [1]:
import os
import os.path as path
import gc
import re
import math
import json

In [69]:
from datetime import datetime
import time

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [3]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..')
DIR_SAVE_PATH = path.join('.')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..
.


In [40]:
# capsule_raw_data.csv
capsule_read = pd.read_csv(path.join(DIR_PATH, 'capsule_raw_data.csv'), low_memory=False)

print(capsule_read.shape)
capsule_read.head()

(167, 12)


,id,thumbnail,user_grade,title,coffeeing_note,company,flavor,bitterness,acidity,body,roasting_point,machine_type
0,1,?,0,Amaretti,Almond Biscuit,Nespresso,?,3,3,3,3,original
1,2,?,0,Bukeela ka Ethiopia*,Fruity / floral,Nespresso,3,1,2,1,1,original
2,3,?,0,Caf? Istanbul,"Intense, spicy",Nespresso,10,?,?,?,?,original
3,4,?,0,Caramelito*,Caramel,Nespresso,6,3,3,3,3,original
4,5,?,0,Ciocattino*,Chocolate,Nespresso,6,3,3,3,3,original


In [55]:
capsule_data = capsule_read.copy()
capsule_data.drop(labels=['id'], axis=1, inplace=True)
capsule_data.head()

,thumbnail,user_grade,title,coffeeing_note,company,flavor,bitterness,acidity,body,roasting_point,machine_type
0,?,0,Amaretti,Almond Biscuit,Nespresso,?,3,3,3,3,original
1,?,0,Bukeela ka Ethiopia*,Fruity / floral,Nespresso,3,1,2,1,1,original
2,?,0,Caf? Istanbul,"Intense, spicy",Nespresso,10,?,?,?,?,original
3,?,0,Caramelito*,Caramel,Nespresso,6,3,3,3,3,original
4,?,0,Ciocattino*,Chocolate,Nespresso,6,3,3,3,3,original


In [56]:
print(capsule_data['thumbnail'].value_counts(dropna=False))

?    167
Name: thumbnail, dtype: int64


In [57]:
# thumbnail 결측치 채우기
capsule_data.replace({'thumbnail':'?'}, 'default_capsule.png', inplace=True)
print(capsule_data['thumbnail'].value_counts(dropna=False))
capsule_data.head(1)

default_capsule.png    167
Name: thumbnail, dtype: int64


,thumbnail,user_grade,title,coffeeing_note,company,flavor,bitterness,acidity,body,roasting_point,machine_type
0,default_capsule.png,0,Amaretti,Almond Biscuit,Nespresso,?,3,3,3,3,original


In [58]:
print(capsule_data['flavor'].value_counts(dropna=False))
print(capsule_data['acidity'].value_counts(dropna=False))
print(capsule_data['roasting_point'].value_counts(dropna=False))

?     48
6     25
4     18
8     16
5     16
9     13
7     13
10     7
11     6
13     2
3      1
12     1
2      1
Name: flavor, dtype: int64
1    63
3    39
2    28
?    26
4     8
5     2
0     1
Name: acidity, dtype: int64
2    50
3    45
4    32
?    19
5    13
1     8
Name: roasting_point, dtype: int64


In [59]:
# 숫자 결측치 채우기
# 단순하게 측정치의 중앙인 3로 결측치 보간한 뒤 10점 만점으로 변환
capsule_data.replace({'acidity':'?', 'bitterness':'?', 'body':'?', 'roasting_point':'?'}, '3', inplace=True)
capsule_data = capsule_data.astype({'acidity':'int', 'bitterness':'int', 'body':'int', 'roasting_point':'int'})
capsule_data[['acidity', 'bitterness', 'body', 'roasting_point']] = capsule_data [['acidity', 'bitterness', 'body', 'roasting_point']] * 2
print(capsule_data['acidity'].value_counts(dropna=False))
capsule_data.head(2)

6     65
2     63
4     28
8      8
10     2
0      1
Name: acidity, dtype: int64


,thumbnail,user_grade,title,coffeeing_note,company,flavor,bitterness,acidity,body,roasting_point,machine_type
0,default_capsule.png,0,Amaretti,Almond Biscuit,Nespresso,?,6,6,6,6,original
1,default_capsule.png,0,Bukeela ka Ethiopia*,Fruity / floral,Nespresso,3,2,4,2,2,original


In [63]:
# 숫자 결측치 채우기 2
# 단순하게 측정치의 중앙인 5로 결측치 보간
capsule_data.replace({'flavor':'?'}, '5', inplace=True)
capsule_data = capsule_data.astype({'flavor':'int'})
capsule_data['flavor'] = capsule_data['flavor'].apply(lambda x : 10 if x > 10 else x)
print(capsule_data['flavor'].value_counts(dropna=False))
capsule_data.head(2)

5     64
6     25
4     18
10    16
8     16
9     13
7     13
3      1
2      1
Name: flavor, dtype: int64


,thumbnail,user_grade,title,coffeeing_note,company,flavor,bitterness,acidity,body,roasting_point,machine_type
0,default_capsule.png,0,Amaretti,Almond Biscuit,Nespresso,5,6,6,6,6,original
1,default_capsule.png,0,Bukeela ka Ethiopia*,Fruity / floral,Nespresso,3,2,4,2,2,original


In [64]:
capsule_data.columns

Index(['thumbnail', 'user_grade', 'title', 'coffeeing_note', 'company',
       'flavor', 'bitterness', 'acidity', 'body', 'roasting_point',
       'machine_type'],
      dtype='object')

In [66]:
capsule_data = capsule_data[[
    'title', 'thumbnail', 'company', 'machine_type', 
    'user_grade', 'coffeeing_note', 'flavor', 'bitterness', 'acidity', 'body', 'roasting_point'
    ]]
capsule_data.head(2)

,title,thumbnail,company,machine_type,user_grade,coffeeing_note,flavor,bitterness,acidity,body,roasting_point
0,Amaretti,default_capsule.png,Nespresso,original,0,Almond Biscuit,5,6,6,6,6
1,Bukeela ka Ethiopia*,default_capsule.png,Nespresso,original,0,Fruity / floral,3,2,4,2,2


In [67]:
## 빠른 데이터 추가를 위해서 중복인 경우 중복 행을 Drop
## 나중에는 중복 데이터의 경우 note는 set으로 유니크한 속성만 남기고, status는 평균 값을 사용
capsule_data.drop_duplicates(subset=['title'], keep='first', inplace=True)
capsule_data['title'].value_counts(dropna=False)

Amaretti                 1
Double Espresso Dolce    1
Bianco Forte             1
Bianco Piccolo           1
Caramel Cookie           1
                        ..
Honduras                 1
Il Caff?                 1
India                    1
Indonesia                1
Ontuoso, Decaf*          1
Name: title, Length: 152, dtype: int64

In [68]:
# 인덱스 재지정
capsule_data.reset_index(drop=True, inplace=True)
capsule_data.head()

,title,thumbnail,company,machine_type,user_grade,coffeeing_note,flavor,bitterness,acidity,body,roasting_point
0,Amaretti,default_capsule.png,Nespresso,original,0,Almond Biscuit,5,6,6,6,6
1,Bukeela ka Ethiopia*,default_capsule.png,Nespresso,original,0,Fruity / floral,3,2,4,2,2
2,Caf? Istanbul,default_capsule.png,Nespresso,original,0,"Intense, spicy",10,6,6,6,6
3,Caramelito*,default_capsule.png,Nespresso,original,0,Caramel,6,6,6,6,6
4,Ciocattino*,default_capsule.png,Nespresso,original,0,Chocolate,6,6,6,6,6


In [70]:
sql_capsule = pd.DataFrame()
sql_capsule['idx'] = capsule_data.index + 1
sql_capsule['created_date'] = datetime(2023,3,31,12).strftime("%Y-%m-%d %H:%M:%S.%f")
sql_capsule['updated_date'] = ""
sql_capsule['name_en'] = capsule_data['title']
sql_capsule['name_ko'] = capsule_data['title']
sql_capsule['summary'] = "아이템 요약 영역 입니다!!!"
sql_capsule['thumbnail'] = capsule_data['thumbnail']
sql_capsule['user_grade'] = 0
sql_capsule.head()

,idx,created_date,updated_date,name_en,name_ko,summary,thumbnail,user_grade
0,1,2023-03-31 12:00:00.000000,,Amaretti,Amaretti,아이템 요약 영역 입니다!!!,default_capsule.png,0
1,2,2023-03-31 12:00:00.000000,,Bukeela ka Ethiopia*,Bukeela ka Ethiopia*,아이템 요약 영역 입니다!!!,default_capsule.png,0
2,3,2023-03-31 12:00:00.000000,,Caf? Istanbul,Caf? Istanbul,아이템 요약 영역 입니다!!!,default_capsule.png,0
3,4,2023-03-31 12:00:00.000000,,Caramelito*,Caramelito*,아이템 요약 영역 입니다!!!,default_capsule.png,0
4,5,2023-03-31 12:00:00.000000,,Ciocattino*,Ciocattino*,아이템 요약 영역 입니다!!!,default_capsule.png,0


In [71]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
sql_capsule.to_csv(path.join(DIR_SAVE_PATH, 'sql_capsule.csv'), sep=',', index=False, encoding='utf-8')

In [73]:
sql_capsule_detail = pd.DataFrame()
sql_capsule_detail['idx'] = capsule_data.index + 1
sql_capsule_detail['created_date'] = datetime(2023,3,31,12).strftime("%Y-%m-%d %H:%M:%S.%f")
sql_capsule_detail['updated_date'] = ""
sql_capsule_detail['company'] = capsule_data['company']
sql_capsule_detail['description'] = "상세 내용 입니다!!!"
sql_capsule_detail['machine_type'] = capsule_data['machine_type']
sql_capsule_detail['origin'] = 'Unknown'
sql_capsule_detail['capsule_idx'] = capsule_data.index + 1
sql_capsule_detail.head()

,idx,created_date,updated_date,company,description,machine_type,origin,capsule_idx
0,1,2023-03-31 12:00:00.000000,,Nespresso,상세 내용 입니다!!!,original,Unknown,1
1,2,2023-03-31 12:00:00.000000,,Nespresso,상세 내용 입니다!!!,original,Unknown,2
2,3,2023-03-31 12:00:00.000000,,Nespresso,상세 내용 입니다!!!,original,Unknown,3
3,4,2023-03-31 12:00:00.000000,,Nespresso,상세 내용 입니다!!!,original,Unknown,4
4,5,2023-03-31 12:00:00.000000,,Nespresso,상세 내용 입니다!!!,original,Unknown,5


In [74]:
# 파일 저장
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
sql_capsule_detail.to_csv(path.join(DIR_SAVE_PATH, 'sql_capsule_detail.csv'), sep=',', index=False, encoding='utf-8')

In [81]:
capsule_data['coffeeing_note'].apply(lambda x : x.lower())

0            almond biscuit
1           fruity / floral
2            intense, spicy
3                   caramel
4                 chocolate
               ...         
147      floral and velvety
148         fruity & cereal
149         smooth & strong
150    dense and harmonious
151      malty and unctuous
Name: coffeeing_note, Length: 152, dtype: object

In [97]:
token = []
word_list_bag = list(capsule_data['coffeeing_note'].apply(lambda x : x.lower().replace(' / ', ', ').replace(' & ', ', ').replace(' and ', ', ').split(', ')))
for word_list in word_list_bag:
    token.extend(word_list)
token = list(set(token))
print(token[:10])
token_df = pd.DataFrame(token)
token_df.to_csv(path.join(DIR_SAVE_PATH, 'capsule_keyword.csv'), sep=',', index=False, encoding='utf-8')

['almond', 'almond biscuit', 'sweet candy', 'caramel', 'flowery', 'woody cedar', 'full-bodied', 'dark roasted', 'malt cereal', 'lively']


In [99]:
# capsule_keyword.csv
token_read = pd.read_csv(path.join(DIR_SAVE_PATH, 'capsule_keyword (2).csv'), low_memory=False)

print(token_read.shape)
token_read.head()

(120, 2)


,en,ko
0,almond,아몬드
1,almond biscuit,"아몬드, 비스킷"
2,sweet candy,사탕
3,caramel,카라멜
4,flowery,플로럴


In [114]:
translate=[]
for word_list in word_list_bag:
    trans=[]
    for word in word_list:
        trans.extend(token_read.loc[token_read['en'] == word]['ko'])
    trans = ', '.join(trans).strip()
    translate.append(trans)
translate

['아몬드, 비스킷',
 '과일, 플로럴',
 '강렬한, 자극적인',
 '카라멜',
 '초코',
 '쌉쌀한, 코코아, 시리얼',
 '달콤한, 시리얼',
 '우디',
 '바닐라, 아몬드, 과일',
 '우디, 자극적인',
 '우디, 자극적인',
 '달콤한, 시리얼, 보리, 구운',
 '자극적인',
 '과일, 시리얼',
 '라즈베리, 시리얼',
 '개암',
 '시리얼',
 '당밀, 자극적인, 우디',
 '레드와인',
 '견과류, 헤이즐넛',
 '바닐라',
 '플로럴, 구운',
 '과일, 플로럴',
 '우디, 자극적인, 코코아',
 '백단나무, 우디, 플로럴',
 '강렬한, 구운, 코코아',
 '강렬한, 구운, 코코아',
 '맥아, 시리얼, 달콤한, 카라멜',
 '달콤한, 카라멜, 팝콘',
 '스모키, 우디, 담배',
 '코코아, 후추',
 '호두, 백단나무, 우디, 허브',
 '구운, 우디',
 '시리얼',
 '카라멜',
 '부드러운, 고소한, 카라멜',
 '부드러운, 고소한, 카라멜, 달콤한',
 '부드러운, 고소한, 초코',
 '다크초코',
 '코코넛, 바닐라',
 '레드와인',
 '자극적인',
 '시리얼, 과일',
 '레몬, 시리얼',
 '레드와인',
 '구운, 스모키',
 '플로럴, 맥아, 시리얼',
 '과일',
 '시리얼, 우디',
 '달콤한, 시리얼',
 '견과류, 과일',
 '시리얼, 코코아',
 '시리얼, 우디',
 '자극적인',
 '우디',
 '과일, 아몬드',
 '후추, 시나몬, 견과류, 고소한',
 '비스킷, 시리얼, 과일',
 '우디, 자극적인',
 '설탕, 체리, 석류',
 '라임, 박하',
 '카라멜',
 '후추, 담배',
 '코코아, 시리얼',
 '자극적인, 과일',
 '달콤한, 허니',
 '우디, 자극적인',
 '풍부한, 견과류',
 '무거운, 강렬한',
 '과일, 달콤한',
 '시리얼, 비스킷',
 '과일, 야채',
 '호박파이, 정향',
 '허브, 자극적인',
 '강렬한, 구운, 과일',
 '강렬한, 구운, 과일',
 '우디, 시리얼

In [115]:
sql_capsule_score = pd.DataFrame()
sql_capsule_score['idx'] = capsule_data.index + 1
sql_capsule_score['created_date'] = datetime(2023,3,31,12).strftime("%Y-%m-%d %H:%M:%S.%f")
sql_capsule_score['updated_date'] = ""
sql_capsule_score['acidity'] = capsule_data['acidity']
sql_capsule_score['balance'] = 5
sql_capsule_score['bitterness'] = capsule_data['bitterness']
sql_capsule_score['body'] = capsule_data['body']
# sql_capsule_score['coffeeing_note'] = capsule_data['coffeeing_note'].apply(lambda x : x.lower().replace(' / ', ', ').replace(' & ', ', ').replace(' and ', ', '))
sql_capsule_score['coffeeing_note'] = translate
sql_capsule_score['flavor'] = capsule_data['flavor']
sql_capsule_score['roasting'] = capsule_data['roasting_point']
sql_capsule_score['capsule_idx'] = capsule_data.index + 1
sql_capsule_score.head()

,idx,created_date,updated_date,acidity,balance,bitterness,body,coffeeing_note,flavor,roasting,capsule_idx
0,1,2023-03-31 12:00:00.000000,,6,5,6,6,"아몬드, 비스킷",5,6,1
1,2,2023-03-31 12:00:00.000000,,4,5,2,2,"과일, 플로럴",3,2,2
2,3,2023-03-31 12:00:00.000000,,6,5,6,6,"강렬한, 자극적인",10,6,3
3,4,2023-03-31 12:00:00.000000,,6,5,6,6,카라멜,6,6,4
4,5,2023-03-31 12:00:00.000000,,6,5,6,6,초코,6,6,5


In [116]:
# 파일 저장|
os.makedirs(DIR_SAVE_PATH, exist_ok=True)
sql_capsule_score.to_csv(path.join(DIR_SAVE_PATH, 'sql_capsule_score.csv'), sep=',', index=False, encoding='utf-8')